In [ ]:
import requests
import pandas as pd
pd.set_option('display.max_rows', None)


# Overpass API URL
overpass_url = "http://overpass-api.de/api/interpreter"

# Overpass query for grocery stores, convenience stores, and schools
query = """
[out:json][timeout:25];
area["name"="Boise"]->.searchArea;

(
  node["shop"="supermarket"](area.searchArea);
  node["shop"="grocery"](area.searchArea);
  node["shop"="convenience"](area.searchArea);
  node["amenity"="school"](area.searchArea);
);
out body;
"""




# Send request to Overpass API
response = requests.get(overpass_url, params={'data': query})
data = response.json()

# List to hold store details
stores = []

# Loop through each element to extract address and type
for element in data['elements']:
    tags = element.get('tags', {})

    # Get the name of the store
    name = tags.get('name', 'N/A')

    # Get coordinates (latitude and longitude)
    lat = element.get('lat')
    lon = element.get('lon')

    # Get address components
    street = tags.get('addr:street', '')
    number = tags.get('addr:housenumber', '')
    city = tags.get('addr:city', '')
    state = tags.get('addr:state', '')
    postcode = tags.get('addr:postcode', '')

    # Build the full address
    address = f"{number} {street}, {city}, {state} {postcode}".strip().strip(',')

    # Determine the type of place (grocery, convenience, etc.)
    place_type = 'N/A'
    if 'shop' in tags:
        if tags['shop'] in ['supermarket', 'grocery', 'convenience']:
            place_type = tags['shop']
    elif 'amenity' in tags and tags['amenity'] == 'school':
        place_type = 'school'

    # Add the store information to the list
    stores.append({
        'Name': name,
        'Address': address,
        'Latitude': lat,
        'Longitude': lon,
        'Type': place_type
    })

# Create DataFrame from the list of stores
df = pd.DataFrame(stores)

# Display the DataFrame
df


,Name,Address,Latitude,Longitude,Type
0,Village Leadership Academy,"219 North Roosevelt Street, Boise, ID 83706",43.606779,-116.233859,school
1,Albertsons,"10500 West Overland Road, Boise, ID 83709",43.591744,-116.313208,supermarket
2,Jacksons,,43.596781,-116.274569,convenience
3,Albertsons,"909 East Parkcenter Boulevard, Boise, ID 83706",43.581120,-116.172155,supermarket
4,Treasure Valley Math and Science Center,,43.680123,-116.279950,school
5,Idaho Statesman,,43.567740,-116.220509,convenience
6,Greenbelt,,43.568289,-116.221985,convenience
7,Natural Grocers,"1195 North Milwaukee Street, Boise, ID 83704",43.615533,-116.285918,supermarket
8,Bodega Boise,"1007 West Main Street, Boise, ID 83702",43.616904,-116.206374,convenience
9,JD's Bodega,"161 North Capitol Boulevard, Boise, ID 83702",43.615598,-116.202083,convenience


In [ ]:

import folium

# Create a map
m = folium.Map(location=[ df.Latitude.median(),df.Longitude.median()], zoom_start=12)

# Add markers for each store to the map
for index, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        tooltip=row['Name'],  
        popup=row['Address'], 
    ).add_to(m)

# Display the map
m


In [ ]:
from time import sleep


# Function for reverse geocoding
def reverse_geocode(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        'format': 'jsonv2',
        'lat': lat,
        'lon': lon,
        'addressdetails': 1
    }
    headers = {
        'User-Agent': 'Mozilla/5.0 (compatible; example script)'
    }

    try:
        response = requests.get(url, params=params, headers=headers)
        if response.status_code == 200:
            data = response.json()
            return data.get('display_name', 'Not found')
        else:
            return f"Error {response.status_code}"
    except Exception as e:
        return str(e)

# Loop through rows and apply reverse geocoding
addresses = []
for i, row in df.iterrows():
    addr = reverse_geocode(row['Latitude'], row['Longitude'])
    addresses.append(addr)
    print(f"Row {i}: {addr}")
    #sleep(1)  # Nominatim rate limit: 1 request per second

# Add the addresses to the DataFrame
df['New_Address'] = addresses

# Display the updated DataFrame
df


Row 0: Village Leadership Academy, 219, North Roosevelt Street, Boise, Ada County, Idaho, 83706, United States
Row 1: Albertsons, 10500, West Overland Road, Boise, Ada County, Idaho, 83709, United States
Row 2: Jacksons, West Camas Street, Boise, Ada County, Idaho, 83708, United States
Row 3: Albertsons, 909, East Parkcenter Boulevard, Southeast Boise, Boise, Ada County, Idaho, 83706, United States
Row 4: Treasure Valley Math and Science Center, 6801, North Gary Lane, Boise, Ada County, Idaho, 83714, United States
Row 5: Idaho Statesman, Upper West Airport Way, Boise, Ada County, Idaho, 83705, United States
Row 6: Greenbelt, Upper West Airport Way, Boise, Ada County, Idaho, 83705, United States
Row 7: Natural Grocers, 1195, North Milwaukee Street, Boise, Ada County, Idaho, 83704, United States
Row 8: Bodega Boise, 1007, West Main Street, Downtown, Boise, Ada County, Idaho, 83702, United States
Row 9: JD's Bodega, 161, North Capitol Boulevard, Downtown, Boise, Ada County, Idaho, 83702, 

,Name,Address,Latitude,Longitude,Type,New_Address
0,Village Leadership Academy,"219 North Roosevelt Street, Boise, ID 83706",43.606779,-116.233859,school,"Village Leadership Academy, 219, North Rooseve..."
1,Albertsons,"10500 West Overland Road, Boise, ID 83709",43.591744,-116.313208,supermarket,"Albertsons, 10500, West Overland Road, Boise, ..."
2,Jacksons,,43.596781,-116.274569,convenience,"Jacksons, West Camas Street, Boise, Ada County..."
3,Albertsons,"909 East Parkcenter Boulevard, Boise, ID 83706",43.581120,-116.172155,supermarket,"Albertsons, 909, East Parkcenter Boulevard, So..."
4,Treasure Valley Math and Science Center,,43.680123,-116.279950,school,"Treasure Valley Math and Science Center, 6801,..."
5,Idaho Statesman,,43.567740,-116.220509,convenience,"Idaho Statesman, Upper West Airport Way, Boise..."
6,Greenbelt,,43.568289,-116.221985,convenience,"Greenbelt, Upper West Airport Way, Boise, Ada ..."
7,Natural Grocers,"1195 North Milwaukee Street, Boise, ID 83704",43.615533,-116.285918,supermarket,"Natural Grocers, 1195, North Milwaukee Street,..."
8,Bodega Boise,"1007 West Main Street, Boise, ID 83702",43.616904,-116.206374,convenience,"Bodega Boise, 1007, West Main Street, Downtown..."
9,JD's Bodega,"161 North Capitol Boulevard, Boise, ID 83702",43.615598,-116.202083,convenience,"JD's Bodega, 161, North Capitol Boulevard, Dow..."
